# **FOR ONE MOVIE**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

**GET REQUEST AND CREATE BEATIFULSOUP OBJECT**

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Jawan_(film)")
print(r)
soup = BeautifulSoup(r.content)

contents = soup.prettify()
print(contents)

**FIND TABLE INFO**

In [ ]:
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")
for row in info_rows:
     print(row.prettify())

**EXTRACT MOVIE NAME**

In [ ]:
movie_name={}
movie_name['title']=info_box.find("th").text
movie_name

{'title': 'Jawan'}

**EXTRACT KEYS AND APPEND TO LIST**

In [ ]:
movie_name_dict = {}
content_key=[]

for idx,i in enumerate(info_box.find_all("th")):
  if idx==0:
    continue
  else:
    content_key.append(i.text)

print(content_key)
print(len(content_key))

['Directed by', 'Screenplay by', 'Dialogues by', 'Story by', 'Produced by', 'Starring', 'Cinematography', 'Edited by', 'Music by', 'Productioncompany', 'Distributed by', 'Release date', 'Running time', 'Country', 'Language', 'Budget', 'Box office']
17


**EXTRACT VALUES AND APPEND TO LIST**

In [ ]:
content_val=[]

for idx,i in enumerate(info_box.find_all("td")):
  if idx==0 or i.text=="\n":
    continue
  else:
    content_val.append(i.text)

print(content_val)
print(len(content_val))

['Atlee', 'AtleeS. Ramanagirivasan\n', 'Sumit Arora', 'Atlee', 'Gauri KhanGaurav Verma', 'Shah Rukh Khan NayantharaVijay Sethupathi', 'G. K. Vishnu', 'Ruben', 'Anirudh Ravichander', 'Red Chillies Entertainment', 'see below', '\n7\xa0September\xa02023\xa0(2023-09-07)\n', '169 minutes (Theatrical release)[1]', 'India', 'Hindi', '₹300 crore[2]', 'est. ₹1,148.32 crore[3]']
17


**MERGE KEYS LIST AND VALUES LIST INTO A DICTIONARY**

In [ ]:
movie_info=result_dict = dict(zip(content_key, content_val))
movie_info


{'Directed by': 'Atlee',
 'Screenplay by': 'AtleeS. Ramanagirivasan\n',
 'Dialogues by': 'Sumit Arora',
 'Story by': 'Atlee',
 'Produced by': 'Gauri KhanGaurav Verma',
 'Starring': 'Shah Rukh Khan NayantharaVijay Sethupathi',
 'Cinematography': 'G. K. Vishnu',
 'Edited by': 'Ruben',
 'Music by': 'Anirudh Ravichander',
 'Productioncompany': 'Red Chillies Entertainment',
 'Distributed by': 'see below',
 'Release date': '\n7\xa0September\xa02023\xa0(2023-09-07)\n',
 'Running time': '169 minutes (Theatrical release)[1]',
 'Country': 'India',
 'Language': 'Hindi',
 'Budget': '₹300 crore[2]',
 'Box office': 'est. ₹1,148.32 crore[3]'}

**COMBINE MOVIE_NAME DICTIONARY AND MOVIE_INFO DICTIONARY**

In [ ]:
movie_all_info={**movie_name,**movie_info}
movie_all_info

{'title': 'Jawan',
 'Directed by': 'Atlee',
 'Screenplay by': 'AtleeS. Ramanagirivasan\n',
 'Dialogues by': 'Sumit Arora',
 'Story by': 'Atlee',
 'Produced by': 'Gauri KhanGaurav Verma',
 'Starring': 'Shah Rukh Khan NayantharaVijay Sethupathi',
 'Cinematography': 'G. K. Vishnu',
 'Edited by': 'Ruben',
 'Music by': 'Anirudh Ravichander',
 'Productioncompany': 'Red Chillies Entertainment',
 'Distributed by': 'see below',
 'Release date': '\n7\xa0September\xa02023\xa0(2023-09-07)\n',
 'Running time': '169 minutes (Theatrical release)[1]',
 'Country': 'India',
 'Language': 'Hindi',
 'Budget': '₹300 crore[2]',
 'Box office': 'est. ₹1,148.32 crore[3]'}


# **FOR ALL MOVIES IN THAT PAGE**


**FUNCTIONS FOR STRING PROCESSSONG - RegEx**

In [ ]:
#FUNCTION 1
from datetime import datetime

def format_date(input_date):
    # Define the correct regular expression pattern
    pattern = r'(?P<day>\d+)\s(?P<month>\w+)\s(?P<year>\d+)\s\((?P<inner_year>\d+-\d+-\d+)\)'
    # Compile the corrected pattern
    compiled_pattern = re.compile(pattern)
    # Search for the pattern in the input date string
    match = compiled_pattern.search(input_date)
    if match:
        # Extract components from the matched groups
        day = match.group('day')
        month = match.group('month')
        year = match.group('year')
        inner_year = match.group('inner_year')
   # Form a datetime object for further processing if needed
        formatted_datetime = datetime.strptime(inner_year, '%Y-%m-%d')
        # Format the date in the desired format
        formatted_date = formatted_datetime.strftime('%d %B %Y')
        return formatted_date
    else:
        return None  # Return None if the pattern is not found
#===========================================================================================================================

#FUNCTION 2

def extract_amount(input_string):
    # Define a regular expression pattern to match the currency format
    pattern = re.compile(r'est\.[₹0-9,.–]+(\s?crore)?|₹[0-9,.]+–[₹0-9,.]+(\s?crore)?|₹[0-9,.]+(\s?crore)?')
    # Search for the pattern in the input string
    match = pattern.search(input_string)

    if match:
        # Extract the matched portion
        extracted_amount = match.group(0)

        # Remove 'est.' part
        extracted_amount = re.sub(r'est\.', '', extracted_amount)

        # Check if there is a range and calculate the average
        range_match = re.search(r'₹([\d,.]+)–[₹]?([\d,.]+)?(\s?crore)?', extracted_amount)
        if range_match:
            lower_limit = float(range_match.group(1).replace(',', ''))
            upper_limit = float(range_match.group(2).replace(',', '')) if range_match.group(2) else lower_limit
            average = (lower_limit + upper_limit) / 2
            return f'₹{average} crore'
        else:
            return extracted_amount
    else:
        return None

#===========================================================================================================================

# FUNCTION 3

def clean_tags(soup):
  for tag in soup.find_all('sup'):
    tag.decompose()
#===========================================================================================================================

# FUNCTION 4
def extract_release_dates(release_dates_str):
    dates = re.findall(r'\b\d{2}\s*[A-Za-z]+\s*\d{4}\b', release_dates_str)
    return dates
#===========================================================================================================================

# FUNCTION 5


def extract_datessss(input_string):
    # Remove non-breaking space characters and extract the date components
    cleaned_string = input_string.replace('\xa0', ' ')
    matches = re.findall(r'\d+ \w+ \d+', cleaned_string)

    if matches:
        extracted_date = matches[0]
        return extracted_date
    else:
        return None
#===========================================================================================================================

# FUNCTION 6
def convert_running_time(running_time_str):
    # Extract the first running time and convert it to an integer
    match = re.search(r'\b(\d+)\b', running_time_str)
    if match:
        return int(match.group(1))
    else:
        return None
#===========================================================================================================================
# FUNCTION 7

def extract_dates_release_dates(input_str):

    # Define a regular expression pattern to match the release dates
    pattern = re.compile(r'(\d{1,2}\s?[a-zA-Z]+\s?\d{4})')

    # Search for the pattern in the input string
    matches = pattern.findall(input_str)

    # Format the matched dates as required
    formatted_dates = [re.sub(r'\xa0', ' ', match) for match in matches]

    return formatted_dates

**FUNCTION THAT RETURNS DICTIONARY HAVING MOVIE_INFO**

In [ ]:
movie_name_dict = {}
movie_name={}

# movie_info={}

def get_info(url):
  content_key=[]
  values_list = []
  resp = requests.get(url)
  # print(resp)
  soup = BeautifulSoup(resp.content)
  clean_tags(soup)
  info_box = soup.find(class_="infobox vevent")
#===========================================================================================================================
  for idx,i in enumerate(info_box.find_all("th")):
    if idx==0 :
      continue
    else:
      content_key.append(i.text.strip())

#===========================================================================================================================
  for idx,i in enumerate(info_box.find_all("td")):
    if idx==0 or i.text=="\n":
      continue
    else:
      values_list.append(i.text.strip())

#===========================================================================================================================

  movie_name['title']=info_box.find("th").text
  #movie_name
#===========================================================================================================================

  movie_info = {key: value for key, value in zip(content_key, values_list)}
  movie_info={**movie_name,**movie_info}
  # movie_info
  return movie_info

**COMBINE BASE_PATH AND RELATIVE_PATH AND GENERATE LINKS FOR ALL MOVIES IN THAT PAGE AND CREATE A LIST OF DICTIONARIES EACH HAVING MOVIE_INFO**

In [ ]:
base_path='https://en.wikipedia.org/'
movie_info_list=[]

r = requests.get("https://en.wikipedia.org/wiki/List_of_Hindi_films_of_2023")
print(r)
soup = BeautifulSoup(r.content)

M=soup.select(".wikitable.sortable i")
for idx, m in enumerate(M):
    relative_path=m.a['href']
    full_path=base_path+relative_path
    title=m.a['title']
    # print(relative_path)
    # print(title)
    print(full_path)
    # print()
    movie_info_list.append(get_info(full_path))


<Response [200]>
https://en.wikipedia.org//wiki/Jawan_(film)
https://en.wikipedia.org//wiki/Pathaan_(film)
https://en.wikipedia.org//wiki/Animal_(2023_film)
https://en.wikipedia.org//wiki/Gadar_2
https://en.wikipedia.org//wiki/Tiger_3
https://en.wikipedia.org//wiki/Dunki_(film)
https://en.wikipedia.org//wiki/Rocky_Aur_Rani_Kii_Prem_Kahaani
https://en.wikipedia.org//wiki/Adipurush
https://en.wikipedia.org//wiki/The_Kerala_Story
https://en.wikipedia.org//wiki/OMG_2


**DATA CLEANING => Call string processing functions defined above**

In [ ]:
# Use format_date and extract_amount functions
for movie_info in movie_info_list:
    # Extract and format the release date
    release_date = movie_info.get('Release date')
    release_dates= movie_info.get('Release dates')
    running_time=movie_info.get('Running time')

    if release_date:
       formatted_release_date = format_date(release_date)
       movie_info['Release date'] = formatted_release_date

    # Extract and format the budget amount and Box office amount
    box_office=movie_info.get('Box office')
    budget = movie_info.get('Budget')

    formatted_budget = extract_amount(budget)
    formatted_box_office= extract_amount(box_office)
    formatted_running_time=convert_running_time(running_time)

    for k,v in movie_info.items():
       if k=='Budget' or k=='Box office':
          uv=extract_amount(v)
          movie_info[k]=uv

    for k,v in movie_info.items():
       if k=='Release dates':
          updated_val=extract_dates_release_dates(v)
          movie_info[k]=updated_val

    movie_info['Budget'] = formatted_budget
    movie_info['Box office'] = formatted_box_office
    movie_info['Running time']=formatted_running_time

    # # Print the updated dictionary
    # print(movie_info)
    # print("-" * 50)


**DISPLAY THE LIST OF DICTIONARIES**

In [ ]:
movie_info_list

[{'title': 'Jawan',
  'Directed by': 'Atlee',
  'Screenplay by': 'AtleeS. Ramanagirivasan',
  'Dialogues by': 'Sumit Arora',
  'Story by': 'Atlee',
  'Produced by': 'Gauri KhanGaurav Verma',
  'Starring': 'Shah Rukh Khan NayantharaVijay Sethupathi',
  'Cinematography': 'G. K. Vishnu',
  'Edited by': 'Ruben',
  'Music by': 'Anirudh Ravichander',
  'Productioncompany': 'Red Chillies Entertainment',
  'Distributed by': 'see below',
  'Release date': '07 September 2023',
  'Running time': 169,
  'Country': 'India',
  'Language': 'Hindi',
  'Budget': '₹300 crore',
  'Box office': '₹1,148.32 crore'},
 {'title': 'Pathaan',
  'Directed by': 'Siddharth Anand',
  'Screenplay by': 'Shridhar Raghavan',
  'Dialogues by': 'Abbas Tyrewala',
  'Story by': 'Siddharth Anand',
  'Produced by': 'Aditya Chopra',
  'Starring': 'Shah Rukh Khan\nDeepika Padukone\nJohn Abraham\nDimple Kapadia\nAshutosh Rana',
  'Cinematography': 'Satchith Paulose',
  'Edited by': 'Aarif Sheikh',
  'Music by': 'Score:Sanchit Ba

**SORT MOVIES BASED ON RUNNING TIME**

In [ ]:
#ascending order of running time
asc_running_time = sorted(movie_info_list, key=lambda x: x['Running time'])
# asc_running_time

**SORT MOVIES BASED ON BOX OFFICE COLLECTION**

In [ ]:
#ascending order of box office collection
def extract_numeric_value(box_office_str):
    match = re.search(r'₹([\d.]+) crore', box_office_str)
    return float(match.group(1)) if match else float('inf')  # Use infinity for entries with no valid numeric value

# Extract the numeric part, convert to float, and sort the list based on the numeric values
asc_box_office = sorted(movie_info_list, key=lambda x: extract_numeric_value(x['Box office']))
#asc_box_office


# **NOTE: GUI IS MADE IN "SPYDER" IDE USING TKINTER**